In [1]:
!pip install tweeterpy

  Obtaining dependency information for tweeterpy from https://files.pythonhosted.org/packages/c3/27/8fbe6fe544880b923dbbae58e6f56066f173e2a8a00f66e9dffafc65f6b7/tweeterpy-1.0.7-py3-none-any.whl.metadata


In [3]:
import json 

In [1]:
from tweeterpy import TweeterPy

In [2]:
twitter = TweeterPy()

2023-09-22 23:46:59,494 [INFO] :: API Updated Successfully.


In [ ]:
twitter.get_liked_tweets('rngland')

In [ ]:
twitter.get_friends('rngland', following=True)

In [5]:
with open("me.json", "w") as a:
    json.dump(twitter.get_user_info('rngland'), a)

2023-09-23 00:03:10,216 [WARNING] :: User is not authenticated.

Successfully Logged In..


KeyboardInterrupt: 